In [12]:
using Pkg
Pkg.add("StatsBase")
Pkg.add("ClassImbalance")
Pkg.add("ROCAnalysis")
using Pkg
using DataFrames
using CSV
using Plots
using GLM
using StatsBase
using Lathe
using MLBase
using ClassImbalance
using ROCAnalysis

   Resolving package versions...
  No Changes to `C:\Users\harsh\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\harsh\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\harsh\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\harsh\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\harsh\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\harsh\.julia\environments\v1.6\Manifest.toml`


In [13]:
ENV["Columns"] = 1000 #enable printing of 1000 columns

1000

In [16]:
df = DataFrame(CSV.File("Churn_Modelling.csv"))
first(df,5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
,Int64,Int64,String,Int64,String,String,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64
1,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,1.01349e5,1
2,2,15647311,Hill,608,Spain,Female,41,1,83807.9,1,0,1,1.12543e5,0
3,3,15619304,Onio,502,France,Female,42,8,1.59661e5,3,1,0,1.13932e5,1
4,4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.6,0
5,5,15737888,Mitchell,850,Spain,Female,43,2,1.25511e5,1,1,1,79084.1,0


In [17]:
first(df,10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
,Int64,Int64,String,Int64,String,String,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64
1,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,1.01349e5,1
2,2,15647311,Hill,608,Spain,Female,41,1,83807.9,1,0,1,1.12543e5,0
3,3,15619304,Onio,502,France,Female,42,8,1.59661e5,3,1,0,1.13932e5,1
4,4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.6,0
5,5,15737888,Mitchell,850,Spain,Female,43,2,1.25511e5,1,1,1,79084.1,0
6,6,15574012,Chu,645,Spain,Male,44,8,1.13756e5,2,1,0,1.49757e5,1
7,7,15592531,Bartlett,822,France,Male,50,7,0.0,2,1,1,10062.8,0
8,8,15656148,Obinna,376,Germany,Female,29,4,1.15047e5,4,1,0,1.19347e5,1
9,9,15792365,He,501,France,Male,44,4,1.42051e5,2,0,1,74940.5,0


In [18]:
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,RowNumber,5000.5,1,5000.5,10000,,,Int64
2,CustomerId,1.56909e7,15565701,1.56907e7,15815690,,,Int64
3,Surname,,Abazu,,Zuyeva,2932,,String
4,CreditScore,650.529,350,652.0,850,,,Int64
5,Geography,,France,,Spain,3,,String
6,Gender,,Female,,Male,2,,String
7,Age,38.9218,18,37.0,92,,,Int64
8,Tenure,5.0128,0,5.0,10,,,Int64
9,Balance,76485.9,0.0,97198.5,2.50898e5,,,Float64


In [20]:
names(df)

14-element Vector{Symbol}:
 :RowNumber
 :CustomerId
 :Surname
 :CreditScore
 :Geography
 :Gender
 :Age
 :Tenure
 :Balance
 :NumOfProducts
 :HasCrCard
 :IsActiveMember
 :EstimatedSalary
 :Exited

In [21]:
countmap(df.Exited)

Dict{Int64, Int64} with 2 entries:
  0 => 7963
  1 => 2037

In [22]:
Lathe.preprocess.OneHotEncode(df,:Geography)
Lathe.preprocess.OneHotEncode(df,:Gender)
select!(df,Not([:RowNumber,:CustomerId,:Surname,:Gender,:Male]))
first(df,5)

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Spain,Germany,Female
,Int64,String,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64,Bool,Bool,Bool,Bool
1,619,France,42,2,0.0,1,1,1,1.01349e5,1,1,0,0,1
2,608,Spain,41,1,83807.9,1,0,1,1.12543e5,0,0,1,0,1
3,502,France,42,8,1.59661e5,3,1,0,1.13932e5,1,1,0,0,1
4,699,France,39,1,0.0,2,0,0,93826.6,0,1,0,0,1
5,850,Spain,43,2,1.25511e5,1,1,1,79084.1,0,0,1,0,1


In [23]:
using Lathe.preprocess: TrainTestSplit
train,test = TrainTestSplit(df,.75)

(7528×14 typename(DataFrame)
│ Row  │ CreditScore │ Geography │ Age   │ Tenure │ Balance   │ NumOfProducts │ HasCrCard │ IsActiveMember │ EstimatedSalary │ Exited │ France │ Spain │ Germany │ Female │
│      │ Int64       │ String    │ Int64 │ Int64  │ Float64   │ Int64         │ Int64     │ Int64          │ Float64         │ Int64  │ Bool   │ Bool  │ Bool    │ Bool   │
├──────┼─────────────┼───────────┼───────┼────────┼───────────┼───────────────┼───────────┼────────────────┼─────────────────┼────────┼────────┼───────┼─────────┼────────┤
│ 1    │ 608         │ Spain     │ 41    │ 1      │ 83807.9   │ 1             │ 0         │ 1              │ 1.12543e5       │ 0      │ 0      │ 1     │ 0       │ 1      │
│ 2    │ 502         │ France    │ 42    │ 8      │ 1.59661e5 │ 3             │ 1         │ 0              │ 1.13932e5       │ 1      │ 1      │ 0     │ 0       │ 1      │
│ 3    │ 699         │ France    │ 39    │ 1      │ 0.0       │ 2             │ 0         │ 0              │ 93

In [24]:

fm = @formula(Exited ~ CreditScore + Age + Tenure + Balance + NumOfProducts +HasCrCard + IsActiveMember + EstimatedSalary +Female + France + Spain)
logit = glm(fm,train,Binomial(),ProbitLink())
#Creating Logistic regression model using Binomial and ProbitLink

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

Exited ~ 1 + CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                        Coef.   Std. Error       z  Pr(>|z|)     Lower 95%    Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)      -1.88615      0.167586     -11.25    <1e-28  -2.21461      -1.55768
CreditScore      -0.000400848  0.000184274   -2.18    0.0296  -0.000762018  -3.96784e-5
Age               0.0430162    0.001686      25.51    <1e-99   0.0397117     0.0463207
Tenure           -0.00783851   0.00616569    -1.27    0.2036  -0.019923      0.00424602
Balance           1.

In [25]:
prediction = predict(logit,test)

2472-element Vector{Union{Missing, Float64}}:
 0.13441027957269924
 0.16312055997869357
 0.028071066778086507
 0.10576384337643277
 0.050904384909257264
 0.16228251312156192
 0.021552378433696825
 0.01958058752299276
 0.03572223969220445
 0.6949527575955071
 0.2522187768523544
 0.06490383515551663
 0.1841944161807767
 ⋮
 0.1456270228601341
 0.06372699111336812
 0.0158213443790309
 0.5404901035192936
 0.04101275266125616
 0.1751168821564963
 0.13701941100688875
 0.5971788490965237
 0.1838453008643957
 0.015698476805728095
 0.21793749315959568
 0.09845067037101483